<a href="https://colab.research.google.com/github/falvojr/dio-lab-open-source/blob/main/PR_Labeler_DIO_Lab_Workflow_Trigger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests

class GitHubConfig:
    """Configurações relacionadas ao GitHub e ao repositório."""
    TOKEN = os.getenv('GITHUB_TOKEN')  # O token vem de uma variável de ambiente
    if not TOKEN:
        raise ValueError("O token do GitHub não foi encontrado. Defina a variável de ambiente 'GITHUB_TOKEN'.")

    OWNER = 'digitalinnovationone'
    REPO = 'dio-lab-open-source'
    HEADERS = {
        'Authorization': f'token {TOKEN}',
        'Accept': 'application/vnd.github.v3+json'
    }

    @staticmethod
    def get_session():
        """Retorna uma sessão configurada para reutilização de conexões HTTP."""
        session = requests.Session()
        session.headers.update(GitHubConfig.HEADERS)
        return session

def obter_prs_abertos():
    """Retorna uma lista de todos os PRs abertos."""
    session = GitHubConfig.get_session()  # Usando a sessão para otimizar as requisições
    next_page = 1
    all_prs = []

    while True:
        try:
            url = f'https://api.github.com/repos/{GitHubConfig.OWNER}/{GitHubConfig.REPO}/pulls?page={next_page}&per_page=100'
            response = session.get(url)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Erro ao recuperar PRs: {e}")
            return []

        prs = response.json()
        all_prs.extend(prs)

        if len(prs) < 100:
            break

        next_page += 1

    print(f"Recuperados {len(all_prs)} PRs abertos no total.")
    return all_prs

def adicionar_label(pr_number):
    """Adiciona a label ao PR especificado."""
    session = GitHubConfig.get_session()  # Usando a sessão
    label_data = {'labels': ['run dio workflow']}
    try:
        url = f'https://api.github.com/repos/{GitHubConfig.OWNER}/{GitHubConfig.REPO}/issues/{pr_number}/labels'
        response = session.post(url, json=label_data)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erro ao adicionar label ao PR {pr_number}: {e}")
        return None

    return response.status_code

def main():
    prs_abertos = obter_prs_abertos()
    prs_abertos.reverse()

    for pr in prs_abertos:
        pr_number = pr['number']
        labels = [label['name'] for label in pr['labels']]

        if 'run dio workflow' not in labels:
            status_code = adicionar_label(pr_number)
            if status_code:
                print(f"Adicionado label ao PR #{pr_number}, Resposta: {status_code}")
        else:
            print(f"PR #{pr_number} já possui o label. Pulando.")

if __name__ == "__main__":
    main()
